In [6]:
import torch
from prompt_toolkit.contrib.regular_languages.regex_parser import tokenize_regex
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

 ### Running the model on a GPU

In [2]:
checkpoint = "TechxGenus/starcoder2-3b-instruct"  

In [3]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    quantization_config=quantization_config,
    device_map ="auto",
    attn_implementation="eager"
)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

C:\Users\Admin\anaconda3\envs\LLM\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--TechxGenus--starcoder2-3b-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors.index.json:   0%|          | 0.00/39.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/7.88k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

Applying chat template

In [4]:
# test_data = dataset["test"]
PROMPT = """### Instruction
{instruction}
### Response
"""

instruction = "write a function that takes a list of integers and returns the sum of all the integers in the list in java"

prompt = PROMPT.format(instruction=instruction)

inputs = tokenizer.encode(prompt, return_tensors="pt")
print(inputs)

tensor([[ 1502, 21052,   222,  1838,   331,   686,   708,  8756,   331,  1168,
           451, 18789,   480,  3235,   341,  3668,   451,  1187,   341, 18789,
           347,   341,  1168,   347,  1401,   222,  1502,  5178,   222]])


In [6]:
decoded_input = tokenizer.decode(inputs[0])
decoded_input

'### Instruction\nwrite a function that takes a list of integers and returns the sum of all the integers in the list in java\n### Response\n'

In [7]:
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=2048)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


### Instruction
write a function that takes a list of integers and returns the sum of all the integers in the list in java
### Response
Here is a simple Java function that takes a list of integers and returns the sum of all the integers in the list:

```java
import java.util.List;

public class Main {
    public static void main(String[] args) {
        List<Integer> list = Arrays.asList(1, 2, 3, 4, 5);
        System.out.println(sumList(list));
    }

    public static int sumList(List<Integer> list) {
        int sum = 0;
        for (Integer number : list) {
            sum += number;
        }
        return sum;
    }
}
```

In this code, we first define a list of integers. Then we call the `sumList` function and print the result. The `sumList` function takes a list of integers as input and returns the sum of all the integers in the list. It does this by initializing a variable `sum` to 0, then iterating over each number in the list, adding it to `sum`, and finally returning `sum`

Without chat prompt

In [45]:
input = "Instruction : write a function that takes a list of integers and returns the sum of all the integers in the list in java. \n Response:"
input_ids = tokenizer(input, return_tensors="pt").to("cuda")
input_ids

{'input_ids': tensor([[     2,  37854,    865,   5598,    476,   1411,    674,   5548,    476,
           1889,    576,  45047,    578,   8753,    573,   2707,    576,    832,
            573,  45047,    575,    573,   1889,    575,   1821, 235265, 235248,
            108,  10567, 235292]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [46]:
outputs = model.generate(input_ids=input_ids['input_ids'], do_sample=True, max_new_tokens=200)
print(tokenizer.decode(outputs[0]))

<bos>Instruction : write a function that takes a list of integers and returns the sum of all the integers in the list in java. 
 Response: 
 
 ```java
public class Main {
  public static int sumOfList(int[] list) {
    int totalSum = 0;
    for (int i = 0; i < list.length; i++) {
      totalSum += list[i];
    }
    return totalSum;
  }
  
  public static void main(String[] args) {
    int[] numbers = {1, 2, 3, 4, 5};
    System.out.println(sumOfList(numbers));
  }
}
```
  
 **The function `sumOfList` does the following:**

1.  **Takes an integer array (list) as input using `int[] list`.**
2.  **Initializes a variable `totalSum` to 0.** This variable will store the cumulative sum. 
3.  **Loops through the


### Inference for LLM for software Model Completion

In [1]:
from datasets import load_dataset

##### Here is the model we use : 

In [2]:
checkpoint = "D:\\LLM\\thesisPractical\\fine_tuned_models\\starcoder2-3b-instruct_complete"

##### The following functions formats the input data to be used in the model

In [4]:
PROMPT = """### Instruction
{input}
### Response
"""

instruction = "write a function that takes a list of integers and returns the sum of all the integers in the list in java"

def format_chat_template_to_print(input):
    return tokenizer(
        PROMPT.format(input=input),
        return_tensors="pt"
    )

##### Load the model and tokenizer

In [7]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    quantization_config=quantization_config,
    device_map ="auto",
    attn_implementation="eager"
)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

##### Load the dataset to test the model

In [8]:
org_path = "D:\\LLM\\thesisPractical\\datasets_for_fine_tuning\\structural_removal_non_contiguous\\processed_4000"

test_dataset_url = org_path + "\\test.jsonl"

data_files = {
    'test' : test_dataset_url
}

dataset = load_dataset('json', data_files=data_files)
test_dataset = dataset['test']

Apply the chat template to the input data

In [9]:
data = test_dataset[0]['input']
text_to_print = format_chat_template_to_print(data)
print(text_to_print)


Token indices sequence length is longer than the specified maximum sequence length for this model (2817 > 2048). Running this sequence through the model will result in indexing errors


{'input_ids': tensor([[ 1502, 21052,   222,  ...,  1502,  5178,   222]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}


In [10]:
output = test_dataset[0]['output']
print(output)

{"nodes":[{"visibility":"PUBLIC_LITERAL","id":1,"eClass":"PackageImport"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"ControlFlow10","name":"ControlFlow10","id":10,"isLeaf":false,"eClass":"ControlFlow"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"ControlFlow18","name":"ControlFlow18","id":17,"isLeaf":false,"eClass":"ControlFlow"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"DecisionNode","name":"DecisionNode","id":25,"isLeaf":false,"eClass":"DecisionNode"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"ActivityFinalNode","name":"ActivityFinalNode","id":26,"isLeaf":false,"eClass":"ActivityFinalNode"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"DecisionNode2","name":"DecisionNode2","id":35,"isLeaf":false,"eClass":"DecisionNode"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"ActivityFinalNode4","name":"ActivityFinalNode4","id":39,"isLeaf":false,"eClass":"ActivityFinalNode"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"ActivityFinalNode5","name":"ActivityFinalNode5","i

Generate the output from the model

In [14]:
outputs = model.generate(input_ids=text_to_print['input_ids'], max_length=4400, do_sample=True)
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded_output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


### Instruction
{"directed":true,"nodes":[{"viewpoint":null,"visibility":"PUBLIC_LITERAL","qualifiedName":"model","name":"model","id":0,"URI":null,"eClass":"Model"},{"isSingleExecution":false,"isReadOnly":false,"visibility":"PUBLIC_LITERAL","qualifiedName":"model::Activity","name":"Activity","id":2,"isActive":false,"isReentrant":true,"isLeaf":false,"isAbstract":false,"isFinalSpecialization":false,"eClass":"Activity"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"ControlFlow","name":"ControlFlow","id":3,"isLeaf":false,"eClass":"ControlFlow"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"ControlFlow2","name":"ControlFlow2","id":4,"isLeaf":false,"eClass":"ControlFlow"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"ControlFlow3","name":"ControlFlow3","id":5,"isLeaf":false,"eClass":"ControlFlow"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"ControlFlow5","name":"ControlFlow5","id":6,"isLeaf":false,"eClass":"ControlFlow"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"ControlFlow6"